In [1]:
import psycopg2
import pandas as pd 
# from sqlalchemy.types import Integer, Text, String, DateTime
import sqlalchemy as s
import matplotlib
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json

with open("config.json") as config_file:
    config = json.load(config_file)

database_connection_string = 'postgres+psycopg2://{}:{}@{}:{}/{}'.format(config['user'], config['password'], config['host'], config['port'], config['database'])

dbschema='augur_data'
engine = s.create_engine(
    database_connection_string,
    connect_args={'options': '-csearch_path={}'.format(dbschema)})

In [2]:
    # Create a dictionary that maps email addresses to github ids
    id_dict = {}
    id_df = pd.DataFrame()
    id_df_query = f"""
            SELECT cntrb_email, cntrb_login from contributors
            """
    id_df = pd.read_sql_query(id_df_query, con=engine)
    id_dict = id_df.set_index('cntrb_email').T.to_dict('list')
#    id_dict = id_df.set_index('cntrb_email').T.to_dict('dict')
    id_df

<ipython-input-2-5da67eba5a49>:8: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  id_dict = id_df.set_index('cntrb_email').T.to_dict('list')


,cntrb_email,cntrb_login
0,pair+gonçalo+kurtis@pivotallabs.com,None
1,markelog@gmail.com,markelog
2,mli@apache.org,liming01
3,pair+cramja+oarap@pivotal.io,None
4,marco.nenciarini@2ndquadrant.it,mnencia
...,...,...
122920,None,pjreddy
122921,None,thall
122922,None,wuguanyu
122923,None,ITman1


In [3]:
print(id_dict['andrew@haines.org.nz'][0])

haines


In [2]:
all_commits = pd.DataFrame()
all_commits_query = f"""
        SELECT DISTINCT(commits.cmt_commit_hash), repo.repo_id, repo.repo_name, repo.repo_path, repo.created_at, 
            commits.cmt_author_email, commits.cmt_author_timestamp from repo, commits
        WHERE 
            repo.repo_id = commits.repo_id
        GROUP BY repo.repo_id, commits.cmt_commit_hash, commits.cmt_author_email, commits.cmt_author_timestamp
        """
all_commits = pd.read_sql_query(all_commits_query, con=engine)
all_commits

,cmt_commit_hash,repo_id,repo_name,repo_path,created_at,cmt_author_email,cmt_author_timestamp
0,5a5a2a46a7928cd2c5e5d517644b214634d5e613,30284,gevent,github.com/carbonblack/,None,denis.bilenko@booking.com,2011-07-31 17:50:46+02:00
1,bcb2c7aa21a7e100f55a99a939d9e546353ae96c,28118,charts,github.com/concourse/,None,sebastien.prudhomme@gmail.com,2019-06-04 14:27:14+02:00
2,5609ffe76c8d22798ef08e987e50a158f39597d7,30573,spring-cloud-contract,github.com/spring-cloud/,None,david_syer@hotmail.com,2016-07-27 10:16:12+02:00
3,f8d5ecb539bdef2b63787e44898375ec6a922245,30046,bitnami-docker-laravel,github.com/bitnami/,None,containers@bitnami.com,2020-11-06 21:51:58+01:00
4,99e6be487a1c6b4015184b792c87c8061f90c81d,30099,bitnami-docker-codiad,github.com/bitnami/,None,containers@bitnami.com,2019-01-21 04:55:09+01:00
...,...,...,...,...,...,...,...
1938680,f4328b41afd390046a49769ce912a34c5e016720,25760,spring-boot,github.com/Spring-projects/,None,pwebb@pivotal.io,2016-11-01 05:48:31+01:00
1938681,55a463a308600050ad6c7a12c77599dcd741d406,30150,bitnami-docker-redis-sentinel,github.com/bitnami/,None,containers@bitnami.com,2019-06-11 06:02:00+02:00
1938682,2a8b540a321eb3e3c568ada73108767f99e90add,25613,credentials-obfuscation,github.com/rabbitmq/,None,michael@clojurewerkz.org,2020-12-18 00:20:27+01:00
1938683,0d667b9a6313a533359c5b68544ee5b3151e29a6,27557,pks-kubo-etcd,github.com/pivotal-cf/,None,cf-mega@pivotal.io,2016-06-22 08:34:00+02:00


In [4]:
test = pd.DataFrame()
test_query = f"""
        SELECT cntrb_login from contributors;
        """
test = pd.read_sql_query(test_query, con=engine)
test

,cntrb_login
0,None
1,markelog
2,liming01
3,None
4,mnencia
...,...
122920,pjreddy
122921,thall
122922,wuguanyu
122923,ITman1


In [9]:
    #by_repo = all_commits.loc[all_commits.groupby('repo_id').cmt_author_timestamp.idxmax()].sort_values('cmt_author_timestamp')
    by_repo = all_commits[all_commits.groupby('repo_id').cmt_author_timestamp.transform('max') == all_commits['cmt_author_timestamp']]
    
    import datetime

    current = datetime.date.today()
    start = current - datetime.timedelta(days=180)
    print(start)
    
    print(by_repo)


2020-06-06
                                  cmt_commit_hash  repo_id  \
55       b353349beb2984325170d17b227ea10d7defcc39    27665   
1783     db773ccf35465c3989b8294e9dc05ceb6203eb42    30359   
2270     841e87a6f6863c8db1b2dcfff1e90b4529c16c6e    30711   
2349     39fe1d98c29e50f4218c0889fa2d41ee8afbb413    27435   
2377     f407a5fdf99eca7c2c41731535a8d476fc10a769    30304   
...                                           ...      ...   
1925946  58283dec188bceb45d0ed3d216496888e2a78689    26475   
1926227  b16a0cc3341da92f241d912f3ea9c13b5bac4737    25471   
1926235  f9ae0151aca5fc0d29e830e9470a2ff8717ba936    27600   
1926409  aecf26849c7e068a56fe1d1111d78147f6543947    30779   
1926522  561e2d4d282e37226ccb7c8cda7f99f17a27d395    26219   

                                repo_name  \
55         docs-xcrypt-service-encryption   
1783       jenkins-github-webhooks-plugin   
2270                   spring-pipeline-m2   
2349                     docs-cloud-cache   
2377               

In [6]:
# This gives us the top contribs per repo - this can be printed to csv file 
i=0
for index, row in by_repo.iterrows():
    repo_link = row.repo_path + row.repo_name
    org = row.repo_path.split("/")[1]
    print(repo_link, org)
    #print(row.repo_path, row.repo_name, row.cmt_author_timestamp, row.cmt_author_email)
    #print(type(row.cmt_author_timestamp))
    top_contribs = all_commits.loc[all_commits['repo_id'] == row.repo_id].cmt_author_email.value_counts()
    if len(top_contribs) > 1:
        print(top_contribs.index[0], top_contribs[0], top_contribs.index[1], top_contribs[1])
    elif len(top_contribs) == 1:
        print(top_contribs.index[0], top_contribs[0])
    elif len(top_contribs) == 0:
        print('None')
    i+=1
    if i == 50:
        break

github.com/rabbitmq/rabbithub rabbitmq
tonyg@rabbitmq.com 80 mikeb@squaremobius.net 2
github.com/vmware/pyvco vmware
yann.hodique@gmail.com 50
github.com/rabbitmq/rabbitmq-ha rabbitmq
matthew@lshift.net 118
github.com/Spring-projects/cloudfoundry-sandbox Spring-projects
peter@cacoethes.co.uk 1 mark@josquin.local 1
github.com/Spring-projects/protocol-fallback-demo Spring-projects
ozhurakousky@pivotal.io 12 bruce.snyder@gmail.com 3
github.com/Spring-projects/nio-sandbox Spring-projects
jon@jbrisbin.com 11
github.com/rabbitmq/rabbitmq-heroku-example rabbitmq
dpw@lshift.net 2 majek@lshift.net 2
github.com/Spring-projects/spring-build Spring-projects
nebhale@nebhale.com 29
github.com/Spring-projects/spring-net-threading Spring-projects
kxu@bn.com 85 gcaprio@1530technologies.com 51
github.com/Spring-projects/insight-plugin-mongodb Spring-projects
stephen@harrison.org 15 jkew@vmware.com 8
github.com/Spring-projects/spring-gemfire-net Spring-projects
mpollack@pivotal.io 39 mpollack@gopivotal.c

# Top Committers by Org

In [9]:
all_commits_query = f"""
        SELECT DISTINCT(commits.cmt_commit_hash), repo.repo_id, repo.repo_group_id, repo.repo_name, repo.repo_path, 
            commits.cmt_author_email, commits.cmt_author_timestamp from repo, commits
        WHERE 
            repo.repo_id = commits.repo_id
            AND commits.cmt_author_email NOT LIKE 'containers%%'
            AND commits.cmt_author_email NOT LIKE 'k8s%%'
            AND commits.cmt_author_email NOT LIKE 'cf-%%'
            AND commits.cmt_author_email NOT LIKE 'ci@%%'
            AND commits.cmt_author_email NOT LIKE 'cb@%%'
            AND commits.cmt_author_email NOT LIKE '%%cve@%%'
            AND commits.cmt_author_email NOT LIKE '%%cves@%%'
        GROUP BY repo.repo_id, commits.cmt_commit_hash, commits.cmt_author_email, commits.cmt_author_timestamp
        ORDER BY repo.repo_id;
        """
all_commits = pd.read_sql_query(all_commits_query, con=engine)
by_repo = all_commits.loc[all_commits.groupby('repo_id').cmt_author_timestamp.idxmax()].sort_values('cmt_author_timestamp')

In [10]:
by_org = all_commits.loc[all_commits.groupby('repo_group_id').cmt_author_timestamp.idxmax()].sort_values('cmt_author_timestamp')
for index, row in by_org.iterrows():
    print 
    top_contribs = all_commits.loc[all_commits['repo_group_id'] == row.repo_group_id].cmt_author_email.value_counts()
    print(row.repo_path, top_contribs.index[0], top_contribs[0], top_contribs.index[1], top_contribs[1], top_contribs.index[2], top_contribs[2])

github.com/GemXD/ swale@snappydata.io 10 gmlewis@google.com 1 ashetkar@users.noreply.github.com 1
github.com/pcfdev-forks/ suraci.alex@gmail.com 4908 justin@specialbusservice.com 3074 crosbymichael@gmail.com 2013
github.com/cfmobile/ david_syer@hotmail.com 921 hone02@gmail.com 723 krishicks@gmail.com 656
github.com/pivotal-education/ droberts@gopivotal.com 110 scottyfred@gmail.com 78 amcginlay@users.noreply.github.com 77
github.com/Pivotal-DataFabric/ jpatel-pivotal@pivotal.io 255 jgronline@gmail.com 171 @Vengers12 146
github.com/pivotal-gss/ bruce@momjian.us 8781 tgl@sss.pgh.pa.us 7844 heikki.linnakangas@iki.fi 1890
github.com/pivotalsoftware/ haoyuan@cs.berkeley.edu 1073 Florian.Schoppmann@emc.com 550 ilayaperumalg@gmail.com 526
github.com/pivotaltracker/ KrauseFx@gmail.com 8712 oliver@cocoanetics.com 2225 jeresig@Archimedes.local 1714
github.com/Pivotal-sg/ raymond.penners@intenct.nl 846 asim@chuhnk.me 224 mcheng.work@gmail.com 207
github.com/Pivotal-Field-Engineering/ bruce@momjian

# Testing getting rid of more None values for gh login

In [14]:
    test_df = pd.DataFrame()
    test_df_query = f"""
            SELECT contributors.cntrb_login, contributors_aliases.canonical_email, 
                contributors_aliases.alias_email from contributors, contributors_aliases
            WHERE contributors_aliases.cntrb_id = contributors.cntrb_id
            """
    test_df = pd.read_sql_query(test_df_query, con=engine)

In [15]:
test_df

,cntrb_login,canonical_email,alias_email
0,squaremo,mikeb@squaremobius.net,mikeb@rabbitmq.com
1,squaremo,mikeb@squaremobius.net,mikeb@lshift.net
2,dumbbell,jean-sebastien.pedron@dumbbell.fr,jean-sebastien@rabbitmq.com
3,michaelklishin,michael@clojurewerkz.org,michael@rabbitmq.com
4,soffes,sam@soff.es,sam@soff.es
...,...,...,...
11473,nleskiw,amergey@gmail.com,nleskiw@gmail.com
11474,None,bartosz.kowalik@comarch.com,bartekviper@gmail.com
11475,None,anadrowski@vmware.com,anadrowski@carbonblack.com
11476,None,mbenhassine@pivotal.io,mahmoud.benhassine@icloud.com


In [136]:
i=0
for item in test_df.iterrows():
#    print(item[1].canonical_email)
#    print(item[1].alias_email)
    canonical_email = item[1].canonical_email
    alias_email = item[1].alias_email
    github_id = item[1].cntrb_login
    try:
        if id_dict[alias_email][0] is None:
            print(alias_email, 'is None, should be', github_id)
            id_dict[alias_email][0] = github_id
            print(alias_email, 'is now', id_dict[alias_email][0])
    except:
        print(alias_email, 'not found')
    try:
        id_dict[canonical_email][0]
    except:
        print(canonical_email, 'not found')
#    if i>100:
#        break
    i+=1

greg@boinc.com is None, should be None
greg@boinc.com is now None
ynvich@gmail.com is None, should be None
ynvich@gmail.com is now None
s@asutp.io is None, should be None
s@asutp.io is now None
kevin.peters@gmx.net is None, should be None
kevin.peters@gmx.net is now None
chollingsworth@pivotal.io is None, should be None
chollingsworth@pivotal.io is now None
dependabot-preview[bot]@users.noreply.github.com is None, should be None
dependabot-preview[bot]@users.noreply.github.com is now None
ci@localhost is None, should be None
ci@localhost is now None
awilkinson@pivotal.io is None, should be None
awilkinson@pivotal.io is now None
anthonycejr@gmail.com is None, should be None
anthonycejr@gmail.com is now None
mlimonczenko@pivotal.io is None, should be None
mlimonczenko@pivotal.io is now None
pair+jarchie@pivotal.io is None, should be None
pair+jarchie@pivotal.io is now None
ci@starkandwayne.com is None, should be None
ci@starkandwayne.com is now None
ci-bot@localhost is None, should be No

tonyg@rabbitmq.com is now None
scvalex@gmail.com is None, should be None
scvalex@gmail.com is now None
emile@rabbitmq.com is None, should be None
emile@rabbitmq.com is now None
marek@rabbitmq.com is None, should be None
marek@rabbitmq.com is now None
matthew@rabbitmq.com is None, should be None
matthew@rabbitmq.com is now None
matthew@lshift.net is None, should be None
matthew@lshift.net is now None
matthias@rabbitmq.com is None, should be None
matthias@rabbitmq.com is now None
matthias@lshift.net is None, should be None
matthias@lshift.net is now None
rob@rabbitmq.com is None, should be None
rob@rabbitmq.com is now None
simon@rabbitmq.com is None, should be None
simon@rabbitmq.com is now None
tim@rabbitmq.com is None, should be None
tim@rabbitmq.com is now None
majek@lshift.net is None, should be None
majek@lshift.net is now None
alexandru@rabbitmq.com is None, should be None
alexandru@rabbitmq.com is now None
ben@lshift.net is None, should be None
ben@lshift.net is now None
0x6e6562@

leo.dos.santos@tasktop.com is now None
tomasz.zarna@tasktop.com is None, should be None
tomasz.zarna@tasktop.com is now None
aeisenberg@vmware.com is None, should be None
aeisenberg@vmware.com is now None
luke@hregnula.local is None, should be None
luke@hregnula.local is now None
Tim.Ysewyn@me.com is None, should be None
Tim.Ysewyn@me.com is now None
Lukasz.Antoniak@gmail.com is None, should be None
Lukasz.Antoniak@gmail.com is now None
eren.avsarogullari@workday.com is None, should be None
eren.avsarogullari@workday.com is now None
erenavsarogullari@yahoo.com is None, should be None
erenavsarogullari@yahoo.com is now None
erenavsarogullari@gmail.com is None, should be None
erenavsarogullari@gmail.com is now None
gregory.bragg@hotmail.com is None, should be None
gregory.bragg@hotmail.com is now None
gbragg@MININT-0B8HP0T.internal.thecommonwell.ca is None, should be None
gbragg@MININT-0B8HP0T.internal.thecommonwell.ca is now None
andypiper@users.noreply.github.com is None, should be Non

thomas@pagerduty.com is None, should be None
thomas@pagerduty.com is now None
kf@splice.com is None, should be None
kf@splice.com is now None
mail@kunday.com is None, should be None
mail@kunday.com is now None
kunday@users.noreply.github.com is None, should be None
kunday@users.noreply.github.com is now None
frederik.mortensen@internetx.com is None, should be None
frederik.mortensen@internetx.com is now None
nmaquet@gmail.com is None, should be None
nmaquet@gmail.com is now None
nicolas@movio.co is None, should be None
nicolas@movio.co is now None
thomas.ploch@flixbus.com is None, should be None
thomas.ploch@flixbus.com is now None
gostrc@gmail.com is None, should be None
gostrc@gmail.com is now None
profiploch@gmail.com is None, should be None
profiploch@gmail.com is now None
34157777+finnplay@users.noreply.github.com is None, should be None
34157777+finnplay@users.noreply.github.com is now None
rcollins@pivotal.io is None, should be None
rcollins@pivotal.io is now None
ryanmattcollin

evan.arnold@gmail.com is None, should be None
evan.arnold@gmail.com is now None
oleg@shaldybin.com is None, should be None
oleg@shaldybin.com is now None
givey@pivotal.io is None, should be None
givey@pivotal.io is now None
david@xspond.com is None, should be None
david@xspond.com is now None
david@collectiveidea.com is None, should be None
david@collectiveidea.com is now None
paul@sitepoint.com is None, should be None
paul@sitepoint.com is now None
paul@annesley.cc is None, should be None
paul@annesley.cc is now None
johnlemberger@gmail.com is None, should be None
johnlemberger@gmail.com is now None
jlemberger@readerplates.com is None, should be None
jlemberger@readerplates.com is now None
oliver.letterer@googlemail.com is None, should be None
oliver.letterer@googlemail.com is now None
oliver.letterer@gmail.com is None, should be None
oliver.letterer@gmail.com is now None
mparker@pivotallabs.com is None, should be None
mparker@pivotallabs.com is now None
pivotal@josephine.den.pivotal.

...

mkb@orthogonal.org is now None
jldec@ciaosoft.com is None, should be None
jldec@ciaosoft.com is now None
stephen.levine@gmail.com is None, should be None
stephen.levine@gmail.com is now None
ytu@pivotal.io is None, should be None
ytu@pivotal.io is now None
aaron.gershman@cerner.com is None, should be None
aaron.gershman@cerner.com is now None
pivotal@Peter-Tran-MacBook-Pro.local is None, should be None
pivotal@Peter-Tran-MacBook-Pro.local is now None
alpha.chen@gmail.com is None, should be None
alpha.chen@gmail.com is now None
lallel121@gmail.com is None, should be None
lallel121@gmail.com is now None
gregg@slackersoft.net is None, should be None
gregg@slackersoft.net is now None
tsaleh@pivotal.io is None, should be None
tsaleh@pivotal.io is now None
aodhner@pivotal.io is None, should be None
aodhner@pivotal.io is now None
paulmeskers@gmail.com is None, should be None
paulmeskers@gmail.com is now None
pivotal@zenhar.local is None, should be None
pivotal@zenhar.local is now None
jpetzol

saiello103@gmail.com is now None
srf@sanger.ac.uk is None, should be None
srf@sanger.ac.uk is now None
yozot@users.noreply.github.com is None, should be None
yozot@users.noreply.github.com is now None
cameronsparr@gmail.com is None, should be None
cameronsparr@gmail.com is now None
simonfr@gmail.com is None, should be None
simonfr@gmail.com is now None
steven@midlink.org is None, should be None
steven@midlink.org is now None
stbarth@cisco.com is None, should be None
stbarth@cisco.com is now None
subhachandra@us101.sjc.aristanetworks.com is None, should be None
subhachandra@us101.sjc.aristanetworks.com is now None
subhachandra@arista.com is None, should be None
subhachandra@arista.com is now None
titilambert@gmail.com is None, should be None
titilambert@gmail.com is now None
thibault.cohen@nuance.com is None, should be None
thibault.cohen@nuance.com is now None
menardorama@gmail.com is None, should be None
menardorama@gmail.com is now None
kurei@axcoto.com is None, should be None
kurei@

sloneg1988@yandex.ru is now None
kostyrev@users.noreply.github.com is None, should be None
kostyrev@users.noreply.github.com is now None
github@auto.thatpanda.com is None, should be None
github@auto.thatpanda.com is now None
richard.maynard@gmail.com is None, should be None
richard.maynard@gmail.com is now None
4426659+wvidana@users.noreply.github.com is None, should be None
4426659+wvidana@users.noreply.github.com is now None
vadice@microsoft.com is None, should be None
vadice@microsoft.com is now None
dalitsohb@gmail.com is None, should be None
dalitsohb@gmail.com is now None
dabanda@microsoft.com is None, should be None
dabanda@microsoft.com is now None
jbielick@users.noreply.github.com is None, should be None
jbielick@users.noreply.github.com is now None
jbielick@gmail.com is None, should be None
jbielick@gmail.com is now None
44974642+rhysemmas@users.noreply.github.com is None, should be None
44974642+rhysemmas@users.noreply.github.com is now None
53451724+ybnelson@users.noreply.g

byt3smith@users.noreply.github.com is None, should be None
byt3smith@users.noreply.github.com is now None
mcgrew@mcgrew-mbp.dhcp.thefacebook.com is None, should be None
mcgrew@mcgrew-mbp.dhcp.thefacebook.com is now None
arg3nbright@gmail.com is None, should be None
arg3nbright@gmail.com is now None
oleksii.kliukin@zalando.de is None, should be None
oleksii.kliukin@zalando.de is now None
alexeyk@me.com is None, should be None
alexeyk@me.com is now None
angel@angelmarquez.com is None, should be None
angel@angelmarquez.com is now None
porkbird@icloud.com is None, should be None
porkbird@icloud.com is now None
sharanmultani@hotmail.com is None, should be None
sharanmultani@hotmail.com is now None
smultani@carbonblack.com is None, should be None
smultani@carbonblack.com is now None
danwachdorf@dwachdorf-mac.bit9.local is None, should be None
danwachdorf@dwachdorf-mac.bit9.local is now None
jgarman@localhost.localdomain is None, should be None
jgarman@localhost.localdomain is now None
jason.

peter@electricpeelsoftware.com is None, should be None
peter@electricpeelsoftware.com is now None
github.com@zurv.de is None, should be None
github.com@zurv.de is now None
pawel@dudek.mobi is None, should be None
pawel@dudek.mobi is now None
nerylix@me.com is None, should be None
nerylix@me.com is now None
matthew.gambill@dowjones.com is None, should be None
matthew.gambill@dowjones.com is now None
simon@zubon.net is None, should be None
simon@zubon.net is now None
simon.cook@trademe.co.nz is None, should be None
simon.cook@trademe.co.nz is now None
tai@tucali.ca is None, should be None
tai@tucali.ca is now None
tai.heng@trubeapp.com is None, should be None
tai.heng@trubeapp.com is now None
taquitos@users.noreply.github.com is None, should be None
taquitos@users.noreply.github.com is now None
taquitos@google.com is None, should be None
taquitos@google.com is now None
tom.wilson@blueapron.com is None, should be None
tom.wilson@blueapron.com is now None
tmtrademarked@users.noreply.github

In [9]:
id_dict['jkew@vmware.com']

[None]

In [20]:
    test_df = pd.DataFrame()
    test_df_query = f"""
            SELECT * from commits
            WHERE cmt_author_email = 'jkew@vmware.com'
            """
    test_df = pd.read_sql_query(test_df_query, con=engine)
    test_df

,cmt_id,repo_id,cmt_commit_hash,cmt_author_name,cmt_author_raw_email,cmt_author_email,cmt_author_date,cmt_author_affiliation,cmt_committer_name,cmt_committer_raw_email,...,cmt_date_attempted,cmt_ght_author_id,cmt_ght_committer_id,cmt_ght_committed_at,tool_source,tool_version,data_source,data_collection_date,cmt_author_timestamp,cmt_committer_timestamp
0,34979925,25713,209268fddbe1381456990c3a86da2ac431ae667a,jkew,jkew@vmware.com,jkew@vmware.com,2011-09-29,VMware,jkew,jkew@vmware.com,...,2011-09-29,None,None,None,'FacadeAugur','0.0.1','git_repository',2020-02-08 17:02:21,2011-09-29 20:35:04+00:00,2011-09-29 20:35:04+00:00
1,34979924,25713,209268fddbe1381456990c3a86da2ac431ae667a,jkew,jkew@vmware.com,jkew@vmware.com,2011-09-29,VMware,jkew,jkew@vmware.com,...,2011-09-29,None,None,None,'FacadeAugur','0.0.1','git_repository',2020-02-08 17:02:21,2011-09-29 20:35:04+00:00,2011-09-29 20:35:04+00:00
2,34979946,25713,22b0cbf342feb82f20d70e5901a88ffe8a73375e,jkew,jkew@vmware.com,jkew@vmware.com,2011-09-29,VMware,jkew,jkew@vmware.com,...,2011-09-29,None,None,None,'FacadeAugur','0.0.1','git_repository',2020-02-08 17:02:21,2011-09-29 23:51:49+00:00,2011-09-29 23:51:49+00:00
3,34979948,25713,22b0cbf342feb82f20d70e5901a88ffe8a73375e,jkew,jkew@vmware.com,jkew@vmware.com,2011-09-29,VMware,jkew,jkew@vmware.com,...,2011-09-29,None,None,None,'FacadeAugur','0.0.1','git_repository',2020-02-08 17:02:21,2011-09-29 23:51:49+00:00,2011-09-29 23:51:49+00:00
4,34979926,25713,209268fddbe1381456990c3a86da2ac431ae667a,jkew,jkew@vmware.com,jkew@vmware.com,2011-09-29,VMware,jkew,jkew@vmware.com,...,2011-09-29,None,None,None,'FacadeAugur','0.0.1','git_repository',2020-02-08 17:02:21,2011-09-29 20:35:04+00:00,2011-09-29 20:35:04+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
302,39851630,25715,5f4ca17d1cc76eff75a19d17801aecb3e879f28b,jkew,jkew@vmware.com,jkew@vmware.com,2012-05-07,VMware,jkew,jkew@vmware.com,...,2012-05-07,None,None,None,'FacadeAugur','0.0.1','git_repository',2020-02-09 04:20:15,2012-05-07 20:02:44+00:00,2012-05-07 20:02:44+00:00
303,39849740,25715,6d7c92d5c14c31538cc9ce824086cd3f338d35db,jkew,jkew@vmware.com,jkew@vmware.com,2012-06-01,VMware,jkew,jkew@vmware.com,...,2012-06-01,None,None,None,'FacadeAugur','0.0.1','git_repository',2020-02-09 04:20:09,2012-06-01 21:22:53+00:00,2012-06-01 21:22:53+00:00
304,39849739,25715,6d7c92d5c14c31538cc9ce824086cd3f338d35db,jkew,jkew@vmware.com,jkew@vmware.com,2012-06-01,VMware,jkew,jkew@vmware.com,...,2012-06-01,None,None,None,'FacadeAugur','0.0.1','git_repository',2020-02-09 04:20:09,2012-06-01 21:22:53+00:00,2012-06-01 21:22:53+00:00
305,39851544,25715,379455c3519ad3e76c8163dc088955461c1145c3,jkew,jkew@vmware.com,jkew@vmware.com,2012-06-01,VMware,jkew,jkew@vmware.com,...,2012-06-01,None,None,None,'FacadeAugur','0.0.1','git_repository',2020-02-09 04:20:14,2012-06-01 21:23:10+00:00,2012-06-01 21:23:10+00:00


In [21]:
    test_df = pd.DataFrame()
    test_df_query = f"""
            SELECT * from contributors
            WHERE cntrb_email = 'jkew@vmware.com'
            """
    test_df = pd.read_sql_query(test_df_query, con=engine)
    test_df

,cntrb_id,cntrb_login,cntrb_email,cntrb_company,cntrb_created_at,cntrb_type,cntrb_fake,cntrb_deleted,cntrb_long,cntrb_lat,...,gh_events_url,gh_received_events_url,gh_type,gh_site_admin,tool_source,tool_version,data_source,data_collection_date,cntrb_full_name,cntrb_last_used
0,329070,None,jkew@vmware.com,None,2011-09-29,None,0,0,None,None,...,None,None,None,None,Contributor Worker,0.0.1,Augur API,2020-03-31 06:29:51,jkew,2012-06-05 00:00:00+00:00


In [15]:
# test to get gh login - not sure why there are so many fewer rows returned?
all_commits = pd.DataFrame()
all_commits_query = f"""
        SELECT DISTINCT(commits.cmt_commit_hash), repo.repo_id, repo.repo_name, repo.repo_path, contributors.cntrb_login, 
            commits.cmt_author_email, commits.cmt_author_timestamp from repo, commits, contributors
        WHERE 
            repo.repo_id = commits.repo_id
            AND contributors.cntrb_canonical = commits.cmt_author_email
        GROUP BY repo.repo_id, commits.cmt_commit_hash, commits.cmt_author_email, commits.cmt_author_timestamp, contributors.cntrb_login
        ORDER BY repo.repo_id;
        """
all_commits = pd.read_sql_query(all_commits_query, con=engine)
all_commits

,cmt_commit_hash,repo_id,repo_name,repo_path,cntrb_login,cmt_author_email,cmt_author_timestamp
0,0097eea9b9cfa0605916fac225513c14a808c910,25430,rmq-0mq,github.com/rabbitmq/,sustrik,sustrik@250bpm.com,2010-09-09 08:49:09+00:00
1,04b6ed16843b336ce6ab18a693c94bbaf669dcee,25430,rmq-0mq,github.com/rabbitmq/,squaremo,mikeb@squaremobius.net,2010-08-13 11:35:42+00:00
2,0a43d1cb58bab9162f50bd4c75bed5bd17bd51ae,25430,rmq-0mq,github.com/rabbitmq/,squaremo,mikeb@squaremobius.net,2010-11-13 18:14:44+00:00
3,0e937a3370247b8f65e7a3eed0f66aea7f255453,25430,rmq-0mq,github.com/rabbitmq/,squaremo,mikeb@squaremobius.net,2010-09-15 11:26:31+00:00
4,1436532e5e2e7381e184b52356dfb3dc3740b636,25430,rmq-0mq,github.com/rabbitmq/,squaremo,mikeb@squaremobius.net,2010-09-10 16:56:12+00:00
...,...,...,...,...,...,...,...
1152012,dab5ef1ceb6c587718cdaf2e46fd47f5d7958190,30332,flask-session,github.com/carbonblack/,fengsp,fsp261@gmail.com,2015-09-17 05:20:15+00:00
1152013,dc7c5ae16d8fd00584860ee322b68d5b0c0a01dc,30332,flask-session,github.com/carbonblack/,fengsp,fsp261@gmail.com,2014-04-25 11:48:29+00:00
1152014,e2b18855384c62d50cc00faf37c7d77c968c2f86,30332,flask-session,github.com/carbonblack/,fengsp,fsp261@gmail.com,2014-04-29 12:02:27+00:00
1152015,e2f676655a7b6669cf5a553a6682fcf5a1a3bda0,30332,flask-session,github.com/carbonblack/,None,skroh@carbonblack.com,2020-04-09 20:27:03+00:00
